In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

import logging

logging.getLogger("py4j").setLevel(logging.DEBUG)


ModuleNotFoundError: No module named 'delta'

In [3]:
# Create SparkSession
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
25/04/22 15:32:39 WARN Utils: Your hostname, NOTEDELL3420 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/22 15:32:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/mnt/c/codigos/spark-delta/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jlsilva01/.ivy2/cache
The jars for the packages stored in: /home/jlsilva01/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6f1fb3cf-c37c-41d3-a003-25c059ec1753;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.2.0/delta-spark_2.12-3.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.2.0!delta-spark_2.12.jar (1261ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.2.0/delta-storage-3.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.2.0!delta-storage.jar (285ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (274ms)
:: resolution report :: resolve 38

In [4]:
spark

In [ ]:
df = spark.read.option("header", True).csv("../../data/listings.csv")

df = df.selectExpr(
    "id as listing_id",
    "host_id",
    "host_name",
    "neighbourhood as city",
    "room_type as property_type",
    "cast(price as double) as price"
)

df.show()


In [5]:
spark.sql(
  """
  CREATE TABLE carro_delta (id INT, placa STRING) USING delta
  """
)

DataFrame[]

In [6]:
spark.sql("select * from carro_delta").show()

25/04/22 15:37:32 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+-----+
| id|placa|
+---+-----+
+---+-----+



In [7]:
from delta.tables import DeltaTable

carro = DeltaTable.forPath(spark, "./spark-warehouse/carro_delta")

In [8]:
carro.history().show()

+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+--------------------+
|version|           timestamp|userId|userName|   operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+--------------------+
|      0|2025-04-22 15:37:...|  NULL|    NULL|CREATE TABLE|{partitionBy -> [...|NULL|    NULL|     NULL|       NULL|  Serializable|         true|              {}|        NULL|Apache-Spark/3.5....|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+--------------------+



In [9]:
spark.sql("INSERT INTO carro_delta VALUES (1, 'XYZ1J34'), (2, 'RLC5B93'), (3, 'ABV1V23')")

DataFrame[]

In [10]:
spark.sql("select * from carro_delta").show()

+---+-------+
| id|  placa|
+---+-------+
|  2|RLC5B93|
|  3|ABV1V23|
|  1|XYZ1J34|
+---+-------+



In [14]:
carro.history().show(truncate=False)

+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                                                                                                                                                                         |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---

In [12]:
spark.sql(
    """
    alter table carro_delta add column marca STRING, modelo STRING, ano INT
    """
)

DataFrame[]

In [13]:
spark.sql(
    """
    select * from carro_delta
    """
).show()

+---+-------+-----+------+----+
| id|  placa|marca|modelo| ano|
+---+-------+-----+------+----+
|  2|RLC5B93| NULL|  NULL|NULL|
|  3|ABV1V23| NULL|  NULL|NULL|
|  1|XYZ1J34| NULL|  NULL|NULL|
+---+-------+-----+------+----+



In [15]:
spark.sql(
    """
    update carro_delta set marca = 'Renault', modelo = 'Sandero', ano = 2021 where id = 1
    """
)

DataFrame[num_affected_rows: bigint]

In [16]:
spark.sql(
    """
    select * from carro_delta
    """
).show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  1|XYZ1J34|Renault|Sandero|2021|
|  2|RLC5B93|   NULL|   NULL|NULL|
|  3|ABV1V23|   NULL|   NULL|NULL|
+---+-------+-------+-------+----+



In [17]:
DeltaTable.isDeltaTable(spark, "spark-warehouse/carro_delta")

True

In [18]:
spark.sql('select * from carro_delta').show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  1|XYZ1J34|Renault|Sandero|2021|
|  2|RLC5B93|   NULL|   NULL|NULL|
|  3|ABV1V23|   NULL|   NULL|NULL|
+---+-------+-------+-------+----+



In [19]:
spark.sql('update carro_delta set marca="GM", modelo="tracker", ano=2020 where id = 2    ')

DataFrame[num_affected_rows: bigint]

In [20]:
spark.sql('update carro_delta set marca="Ford", modelo="EcoSport", ano=2022 where id = 3    ')

DataFrame[num_affected_rows: bigint]

In [21]:
spark.sql('select * from carro_delta').show()

+---+-------+-------+--------+----+
| id|  placa|  marca|  modelo| ano|
+---+-------+-------+--------+----+
|  1|XYZ1J34|Renault| Sandero|2021|
|  3|ABV1V23|   Ford|EcoSport|2022|
|  2|RLC5B93|     GM| tracker|2020|
+---+-------+-------+--------+----+



In [22]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [23]:
spark.sql('describe HISTORY carro_delta').show(truncate=False);

+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                                                                                                                                           

In [ ]:
print("Executando DELETE para remover o registro com id = 3...")
spark.sql('DELETE FROM carro_delta WHERE id = 3')
print("DELETE executado com sucesso.")

In [ ]:
print("Verificando os dados na tabela após o DELETE:")
spark.sql('SELECT * FROM carro_delta').show()